In [1]:
import random
import torch
import numpy as np
def same_seeds(seed):
    random.seed(seed) 
    np.random.seed(seed)  
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed) 
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(2526)

In [2]:
from Preprocessing import load_data
train_sentences, train_labels = load_data('train.txt')
valid_sentences, valid_labels = load_data('valid.txt')
test_sentences, test_labels = load_data('test.txt')
valid_sentences = valid_sentences + test_sentences
valid_labels = valid_labels + test_labels

# TorchText分割

In [3]:
from Preprocessing import torchText
# TorchText分割
token_voc, label_voc = torchText(train_sentences + valid_sentences, 
                                 train_labels + valid_labels, 
                                 specials = ('<PAD>', '<UNK>'))


# 建立必備資訊
PAD_IDX = token_voc.get_stoi()['<PAD>']
O_IDX = label_voc.get_stoi()['O']
INPUT_DIM = len(token_voc)
OUTPUT_DIM = len(label_voc)
LABEL_CLASSES = label_voc.get_stoi().keys()
embedding_dim = 300

# Wor2Vec分割

In [4]:
from Preprocessing import pre_trained_model


token_voc, label_voc, pretrained_emb = pre_trained_model('word2vec-google-news-300',
                                                         train_sentences + valid_sentences, 
                                                         train_labels + valid_labels, 
                                                         specials = ('<PAD>', '<UNK>'))

PAD_IDX = token_voc.get_stoi()['<PAD>']
O_IDX = label_voc.get_stoi()['O']
INPUT_DIM = len(token_voc)
OUTPUT_DIM = len(label_voc)
LABEL_CLASSES = label_voc.get_stoi().keys()
embedding_dim = pretrained_emb.shape[1]

C:\Users\user\Desktop\iThome2023-learn-NLP-in-30-days\【Day 17】解析詞嵌入預訓練模型的奧秘(終)\Preprocessing.py:65: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:264.)
  pretrained_emb = torch.tensor(pretrained_voc)


# GloVe分割

In [11]:
from Preprocessing import pre_trained_model
token_voc, label_voc, pretrained_emb = pre_trained_model('glove-wiki-gigaword-100',
                                                         train_sentences + valid_sentences, 
                                                         train_labels + valid_labels, 
                                                         specials = ('<PAD>', '<UNK>'))

PAD_IDX = token_voc.get_stoi()['<PAD>']
O_IDX = label_voc.get_stoi()['O']
INPUT_DIM = len(token_voc)
OUTPUT_DIM = len(label_voc)
LABEL_CLASSES = label_voc.get_stoi().keys()
embedding_dim = pretrained_emb.shape[1]

# fastText分割

In [3]:
from Preprocessing import pre_trained_model

token_voc, label_voc, pretrained_emb = pre_trained_model('fasttext-wiki-news-subwords-300',
                                                         train_sentences + valid_sentences, 
                                                         train_labels + valid_labels, 
                                                         specials = ('<PAD>', '<UNK>'))

PAD_IDX = token_voc.get_stoi()['<PAD>']
O_IDX = label_voc.get_stoi()['O']
INPUT_DIM = len(token_voc)
OUTPUT_DIM = len(label_voc)
LABEL_CLASSES = label_voc.get_stoi().keys()
embedding_dim = pretrained_emb.shape[1]

C:\Users\user\Desktop\NER\Preprocessing.py:66: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:248.)
  pretrained_emb = torch.tensor(pretrained_voc)


In [5]:
from Preprocessing import tokens2nums

#建立資料集
x_train, y_train = tokens2nums(train_sentences, train_labels, token_voc, label_voc)
x_valid, y_valid = tokens2nums(valid_sentences, valid_labels, token_voc, label_voc)

In [6]:
from torch.nn.utils.rnn import pad_sequence

x_train, y_train = pad_sequence(x_train, padding_value=PAD_IDX, batch_first=True), \
                   pad_sequence(y_train, padding_value=PAD_IDX, batch_first=True)

x_valid, y_valid = pad_sequence(x_valid, padding_value=PAD_IDX, batch_first=True), \
                   pad_sequence(y_valid, padding_value=PAD_IDX, batch_first=True)

In [7]:
from torch.utils.data import Dataset, DataLoader
import torch
class NERDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y
          
    def __getitem__(self, index):
        return self.x[index], self.y[index]
       
    def __len__(self):
        return len(self.x)

trainset = NERDataset(x_train, y_train)
validset = NERDataset(x_valid, y_valid)

train_loader = DataLoader(trainset, batch_size = 1024, shuffle = True, num_workers = 0, pin_memory = True)
valid_loader = DataLoader(validset, batch_size = 1024, shuffle = True, num_workers = 0, pin_memory = True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
import torch.nn as nn
import torch.optim as optim

class NERModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size):
        super(NERModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = PAD_IDX)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.hidden2tag = nn.Linear(hidden_dim * 2, tagset_size)

    def forward(self, sentence):
        embeds = self.embedding(sentence)
        lstm_out, _ = self.lstm(embeds)
        tag_space = self.hidden2tag(lstm_out)
        return tag_space


hidden_dim = 100
model = NERModel(INPUT_DIM, embedding_dim, hidden_dim, OUTPUT_DIM).to(device)
try:
    model.embedding.weight.data.copy_(pretrained_emb)
except:
    pass
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

In [10]:
from tqdm import tqdm
import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score

def train(epoch):
    train_loss, train_acc = 0, 0
    train_pbar = tqdm(train_loader, position=0, leave=True)
    
    model.train()
    all_preds, all_true = [], []
    for input_datas in train_pbar: 
        features, labels = [i.to(device) for i in input_datas] 
        optimizer.zero_grad()  
        outputs = model(features)
        loss = criterion(outputs.view(-1, OUTPUT_DIM), labels.view(-1)) 
        loss.backward() 
        optimizer.step() 
        
        train_pbar.set_description(f'Train Epoch {epoch}')
        train_pbar.set_postfix({'loss':f'{loss:.3f}'}) 
        
        _, preds = torch.max(outputs, dim = 2)
        train_loss += loss.item()  # 模型總損失
        
        all_preds.extend(preds.cpu().numpy())
        all_true.extend(labels.cpu().numpy())
        
    all_true = np.concatenate(all_true, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)
    idx = all_true != O_IDX
    
    return f1_score(all_true[idx], all_preds[idx], average = 'micro'), train_acc/len(trainset) 


def valid(epoch):
    valid_loss, valid_acc = 0, 0
    valid_pbar = tqdm(valid_loader, position=0, leave=True)
    all_preds, all_true = [], []
    model.eval()
    with torch.no_grad(): 
        for input_datas in valid_pbar:
            features, labels = [i.to(device) for i in input_datas]
            outputs = model(features) 
            loss = criterion(outputs.view(-1, OUTPUT_DIM), labels.view(-1))

            valid_pbar.set_description(f'Valid Epoch {epoch}')
            valid_pbar.set_postfix({'loss':f'{loss:.3f}'})

            _, preds = torch.max(outputs, dim = 2)
            valid_loss += loss.item()

            all_preds.extend(preds.cpu().numpy())
            all_true.extend(labels.cpu().numpy())
            
        all_true = np.concatenate(all_true, axis=0)
        all_preds = np.concatenate(all_preds, axis=0)
        idx = all_true != O_IDX
        
    return f1_score(all_true[idx], all_preds[idx], average = 'micro'), valid_acc/len(validset)

        
def show_training_loss(loss_record):
    train_loss, valid_loss = [i for i in loss_record.values()]
    
    plt.plot(train_loss)
    plt.plot(valid_loss)
    #標題
    plt.title('Result')
    #y軸標籤
    plt.ylabel('Loss')
    #x軸標籤
    plt.xlabel('Epoch')
    #顯示折線的名稱
    plt.legend(['train', 'valid'], loc='upper left')
    #顯示折線圖
    plt.show()

In [11]:
import numpy as np
epochs = 10000                              # 訓練次數
early_stopping = 20                         # 模型訓練幾次沒進步就停止
stop_cnt = 0                                # 計數模型是否有進步的計數器
model_path = 'model.ckpt'                   # 模型存放路徑
show_loss = False                           # 是否顯示訓練折線圖
best_f1 = 0                                 # 最佳的準確率
loss_record = {'train':[], 'valid':[]}      # 訓練紀錄

for epoch in range(epochs):   
    train_f1, train_loss= train(epoch)
    valid_f1, valid_loss = valid(epoch)
    
    loss_record['train'].append(train_loss)
    loss_record['valid'].append(valid_loss)
    
    # 儲存最佳的模型權重
    if best_f1 < valid_f1:
        best_f1 = valid_f1
        torch.save(model.state_dict(), model_path)
        print(f'Saving Model With F1 {best_f1:.5f}')
        stop_cnt = 0
    else:
        stop_cnt+=1
    
    # Early stopping
    if stop_cnt == early_stopping:
        output = "Model can't improve, stop training"
        print('-' * (len(output)+2))
        print(f'|{output}|')
        print('-' * (len(output)+2))
        break
        
    print(f'Train Loss: {train_loss:.5f} Train F1: {train_f1:.5f}', end='| ')
    print(f'Valid Loss: {valid_loss:.5f} Valid F1: {valid_f1:.5f}', end='| ')
    print(f'Best F1: {best_f1:.5f}', end ='\n\n')

if show_loss:
    show_training_loss(loss_record)

Valid Epoch 0: 100%|█████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 37.37it/s, loss=1.212]


Train Loss: 0.00000 Train F1: 0.04885| Valid Loss: 0.00000 Valid F1: 0.00000| Best F1: 0.00000



Valid Epoch 1: 100%|█████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 49.83it/s, loss=0.142]


Train Loss: 0.00000 Train F1: 0.00000| Valid Loss: 0.00000 Valid F1: 0.00000| Best F1: 0.00000



Valid Epoch 2: 100%|█████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 49.83it/s, loss=0.127]


...

Train Loss: 0.00000 Train F1: 0.99386| Valid Loss: 0.00000 Valid F1: 0.74765| Best F1: 0.74987



Valid Epoch 95: 100%|████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 44.95it/s, loss=0.030]


Train Loss: 0.00000 Train F1: 0.99413| Valid Loss: 0.00000 Valid F1: 0.74586| Best F1: 0.74987



Valid Epoch 96: 100%|████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 44.30it/s, loss=0.025]

------------------------------------
|Model can't improve, stop training|
------------------------------------
